In [ ]:
# Wizualizacja Danych, Lab 5 - Jan Banot
# Zadanie 1
import seaborn as sns
import matplotlib.pyplot as plt

tips = sns.load_dataset("tips")
plt.figure(figsize=(8, 6))

sns.scatterplot(x='total_bill', y='tip', data=tips, palette='viridis')
plt.title('Total Bill vs Tip')
plt.show()

In [ ]:
# Zadanie 2
plt.figure(figsize=(10, 7))
sns.boxplot(x='day', y='total_bill', data=tips)

sns.swarmplot(x='day', y='total_bill', data=tips, color='black', alpha=0.5, size=3)
plt.title('Boxplot with Swarmplot')
plt.show()

In [ ]:
# Zadanie 3
iris = sns.load_dataset("iris")

sns.pairplot(iris, hue='species', markers=['o', 's', 'D'], palette='husl')
plt.show()

In [ ]:
# Zadanie 4
titanic = sns.load_dataset("titanic")
g = sns.FacetGrid(titanic, col='survived')

g.map(plt.hist, 'age', bins=20)
plt.show()

In [ ]:
# Zadanie 5
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

p = figure(tools="hover", title="Scatter Plot with Hover", tooltips="@x, @y")
p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=15)

p.hover.tooltips = [
    ("Index", "$index"),
    ("(x, y)", "($x, $y)"),
    ("Radius", "@radius")
]
show(p)

In [ ]:
# Zadanie 6
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap

fruits = ["Apples", "Oranges", "Pears", "Bananas"]
counts = [10, 20, 14, 22]
source = ColumnDataSource(data=dict(fruits=fruits, counts=counts))

p = figure(
    x_range=fruits, title="Interactive Bar Plot", toolbar_location=None, tools=""
)

p.vbar(
    x="fruits",
    top="counts",
    width=0.9,
    source=source,
    fill_color=factor_cmap(
        "fruits", palette=["#c9d9d3", "#718dbf", "#e84d60", "#ddb7b1"], factors=fruits
    ),
)
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.legend.orientation = "horizontal"
p.legend.location = "top_left"
show(p)

In [ ]:
# Zadanie 7
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bokeh.models import HoverTool

dates = [datetime.now() - timedelta(days=x) for x in range(30)]
dates.reverse()
stock_prices = np.random.uniform(100, 200, 30)

df = pd.DataFrame({"date": dates, "price": stock_prices})

z = np.polyfit(range(len(df)), df["price"], 1)
p_trend = np.poly1d(z)
df["trend"] = p_trend(range(len(df)))

source = ColumnDataSource(df)

p = figure(
    x_axis_type="datetime",
    title="Stock Price Time Series",
    width=900,
    height=400,
    tools="pan,wheel_zoom,box_zoom,reset",
)

p.line(
    "date",
    "price",
    source=source,
    line_width=2,
    color="navy",
    alpha=0.8,
    legend_label="Stock Price",
)

p.circle("date", "price", source=source, size=6, color="navy", alpha=0.5)

p.line(
    "date",
    "trend",
    source=source,
    line_width=2,
    color="red",
    alpha=0.6,
    line_dash="dashed",
    legend_label="Trend",
)

hover = HoverTool(
    tooltips=[
        ("Date", "@date{%F}"),
        ("Price", "@price{0.2f}"),
    ]
)
hover.formatters = {"@date": "datetime"}
p.add_tools(hover)

p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "Stock Price ($)"
p.legend.location = "top_left"
p.legend.click_policy = "hide"

show(p)

In [ ]:
# Zadanie 8
from bokeh.models import HoverTool, CustomJS, LabelSet

categories = ["Elektronika", "Odzież", "Żywność", "Meble"]
sales = np.random.randint(50, 500, len(categories))

source = ColumnDataSource(
    data=dict(
        categories=categories,
        sales=sales,
        colors=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"],
    )
)

p = figure(
    x_range=categories,
    title="Liczba Sprzedanych Produktów według Kategorii",
    width=800,
    height=500,
    toolbar_location="above",
    tools="tap,reset",
)

bars = p.vbar(
    x="categories",
    top="sales",
    width=0.7,
    source=source,
    color="colors",
    alpha=0.8,
    line_color="white",
    line_width=2,
)

hover = HoverTool(
    tooltips=[
        ("Kategoria", "@categories"),
        ("Sprzedaż", "@sales sztuk"),
    ]
)
p.add_tools(hover)

source.selected.js_on_change(
    "indices",
    CustomJS(
        args=dict(source=source),
        code="""
    const indices = source.selected.indices;
    if (indices.length > 0) {
        const idx = indices[0];
        const cat = source.data['categories'][idx];
        const val = source.data['sales'][idx];
        alert('Wybrana kategoria: ' + cat + '\\nSprzedaż: ' + val + ' sztuk');
    }
""",
    ),
)

p.xaxis.axis_label = "Kategoria Produktu"
p.yaxis.axis_label = "Liczba Sprzedanych Sztuk"
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = 0.785

labels = LabelSet(
    x="categories",
    y="sales",
    text="sales",
    y_offset=5,
    text_align="center",
    text_baseline="bottom",
    source=source,
    text_font_size="12pt",
    text_color="black",
)
p.add_layout(labels)

show(p)

In [ ]:
# Zadanie 9
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column

months = [
    "Sty",
    "Lut",
    "Mar",
    "Kwi",
    "Maj",
    "Cze",
    "Lip",
    "Sie",
    "Wrz",
    "Paź",
    "Lis",
    "Gru",
]
revenues = np.random.randint(8000, 15000, 12)
expenses = np.random.randint(5000, 10000, 12)


def calculate_waterfall(revenues, expenses):
    net_profit = revenues - expenses
    cumulative = np.cumsum(net_profit)

    # Przygotowanie danych dla waterfall
    bars_data = []
    colors = []

    start_value = 0
    for i in range(len(months)):
        profit = net_profit[i]
        color = "#2ca02c" if profit >= 0 else "#d62728"
        bars_data.append(
            {
                "month": months[i],
                "bottom": start_value,
                "top": start_value + profit,
                "profit": profit,
                "cumulative": cumulative[i],
                "revenue": revenues[i],
                "expense": expenses[i],
            }
        )
        colors.append(color)
        start_value += profit

    return bars_data, colors


bars_data, colors = calculate_waterfall(revenues, expenses)

# Przygotowanie danych dla Bokeh
source = ColumnDataSource(
    data={
        "months": [d["month"] for d in bars_data],
        "bottom": [d["bottom"] for d in bars_data],
        "top": [d["top"] for d in bars_data],
        "profit": [d["profit"] for d in bars_data],
        "cumulative": [d["cumulative"] for d in bars_data],
        "revenue": [d["revenue"] for d in bars_data],
        "expense": [d["expense"] for d in bars_data],
        "colors": colors,
    }
)

p = figure(
    x_range=months,
    title="Wykres Kaskadowy - Przychody i Wydatki (PLN)",
    width=1000,
    height=500,
    toolbar_location="above",
    tools="hover,pan,wheel_zoom,box_zoom,reset",
)

p.vbar(
    x="months",
    bottom="bottom",
    top="top",
    width=0.7,
    source=source,
    color="colors",
    alpha=0.8,
    line_color="white",
    line_width=2,
)

p.line(
    "months",
    "cumulative",
    source=source,
    line_width=3,
    color="navy",
    alpha=0.6,
    legend_label="Zysk kumulatywny",
)
p.circle("months", "cumulative", source=source, size=8, color="navy", alpha=0.6)

hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Miesiąc", "@months"),
    ("Przychody", "@revenue{0,0} PLN"),
    ("Wydatki", "@expense{0,0} PLN"),
    ("Zysk netto", "@profit{0,0} PLN"),
    ("Zysk kumulatywny", "@cumulative{0,0} PLN"),
]

p.xaxis.axis_label = "Miesiąc"
p.yaxis.axis_label = "Kwota (PLN)"
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.click_policy = "hide"

revenue_slider = Slider(
    start=5000,
    end=20000,
    value=10000,
    step=500,
    title="Mnożnik przychodów (%)",
    width=900,
)
expense_slider = Slider(
    start=5000,
    end=20000,
    value=10000,
    step=500,
    title="Mnożnik wydatków (%)",
    width=900,
)

callback = CustomJS(
    args=dict(source=source, rev_slider=revenue_slider, exp_slider=expense_slider),
    code="""
    const data = source.data;
    const rev_mult = rev_slider.value / 10000;
    const exp_mult = exp_slider.value / 10000;

    let cumulative = 0;

    for (let i = 0; i < data['months'].length; i++) {
        const revenue = data['revenue'][i] * rev_mult;
        const expense = data['expense'][i] * exp_mult;
        const profit = revenue - expense;

        data['bottom'][i] = cumulative;
        data['top'][i] = cumulative + profit;
        data['profit'][i] = profit;

        cumulative += profit;
        data['cumulative'][i] = cumulative;

        // Zmiana koloru w zależności od zysku
        data['colors'][i] = profit >= 0 ? '#2ca02c' : '#d62728';
    }

    source.change.emit();
""",
)

revenue_slider.js_on_change("value", callback)
expense_slider.js_on_change("value", callback)

layout = column(p, revenue_slider, expense_slider)
show(layout)

In [ ]:
# Zadanie 10
from bokeh.transform import cumsum
from math import pi

categories = ["Wynajem", "Żywność", "Transport", "Rozrywka"]
percentages = np.random.uniform(10, 40, len(categories))
percentages = percentages / percentages.sum() * 100

total_budget = 5000
values = (percentages / 100) * total_budget

data = pd.DataFrame(
    {
        "category": categories,
        "percentage": percentages,
        "value": values,
        "angle": percentages / 100 * 2 * pi,
        "color": ["#3498db", "#2ecc71", "#f39c12", "#e74c3c"],
    }
)

data["angle"] = data["percentage"] / 100 * 2 * pi

p = figure(
    title="Udział Kategorii Wydatków w Budżecie",
    toolbar_location=None,
    tools="hover",
    tooltips="@category: @percentage{0.2f}% (@value{0,0} PLN)",
    width=600,
    height=600,
)

p.wedge(
    x=0,
    y=0,
    radius=0.4,
    start_angle=cumsum("angle", include_zero=True),
    end_angle=cumsum("angle"),
    line_color="white",
    fill_color="color",
    legend_field="category",
    source=data,
)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
p.legend.location = "right"
p.legend.label_text_font_size = "12pt"

show(p)

In [ ]:
# Zadanie 11
from bokeh.models import (
    LinearColorMapper,
    ColorBar,
    BasicTicker,
    PrintfTickFormatter,
    Select,
    CustomJS,
)
from bokeh.palettes import RdYlGn11
from bokeh.layouts import column

students = [f"Uczeń {i + 1}" for i in range(10)]
subjects = ["Matematyka", "Fizyka", "Chemia", "Biologia"]

np.random.seed(42)
scores_data = {}
for subject in subjects:
    scores_data[subject] = np.random.randint(50, 100, len(students))

df_scores = pd.DataFrame(scores_data, index=students)

students_list = []
subjects_list = []
scores_list = []
colors_list = []

for student in students:
    for subject in subjects:
        students_list.append(student)
        subjects_list.append(subject)
        scores_list.append(df_scores.loc[student, subject])

source = ColumnDataSource(
    data=dict(students=students_list, subjects=subjects_list, scores=scores_list)
)

original_data = dict(source.data)

colors = RdYlGn11
mapper = LinearColorMapper(palette=colors, low=50, high=100)

p = figure(
    title="Mapa Ciepła - Wyniki Egzaminów",
    x_range=subjects,
    y_range=list(reversed(students)),
    width=700,
    height=600,
    toolbar_location="above",
    tools="hover,save,reset",
)

p.rect(
    x="subjects",
    y="students",
    width=1,
    height=1,
    source=source,
    fill_color={"field": "scores", "transform": mapper},
    line_color="white",
)

hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Uczeń", "@students"),
    ("Przedmiot", "@subjects"),
    ("Wynik", "@scores"),
]

color_bar = ColorBar(
    color_mapper=mapper,
    major_label_text_font_size="10pt",
    ticker=BasicTicker(desired_num_ticks=10),
    formatter=PrintfTickFormatter(format="%d"),
    label_standoff=12,
    border_line_color=None,
    location=(0, 0),
)
p.add_layout(color_bar, "right")

p.xaxis.axis_label = "Przedmiot"
p.yaxis.axis_label = "Uczeń"
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.xaxis.major_label_orientation = 0.785

subject_options = ["Wszystkie"] + subjects
select = Select(
    title="Filtruj według przedmiotu:",
    value="Wszystkie",
    options=subject_options,
    width=300,
)

callback = CustomJS(
    args=dict(
        source=source,
        original=original_data,
        subjects=subjects,
        students=students,
        select=select,
    ),
    code="""
    const selected = select.value;
    const orig_students = original['students'];
    const orig_subjects = original['subjects'];
    const orig_scores = original['scores'];

    if (selected === 'Wszystkie') {
        source.data = original;
    } else {
        const new_students = [];
        const new_subjects = [];
        const new_scores = [];

        for (let i = 0; i < orig_students.length; i++) {
            if (orig_subjects[i] === selected) {
                new_students.push(orig_students[i]);
                new_subjects.push(orig_subjects[i]);
                new_scores.push(orig_scores[i]);
            }
        }

        source.data = {
            students: new_students,
            subjects: new_subjects,
            scores: new_scores
        };
    }

    source.change.emit();
""",
)

select.js_on_change("value", callback)

layout = column(select, p)
show(layout)